In [4]:
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import time
from selenium.webdriver.firefox.options import Options as FirefoxOptions
from selenium.webdriver.edge.options import Options as eoptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from lxml import etree

firefox_options = FirefoxOptions()

firefox_options.add_argument('--headless=false') 

Eoptions = eoptions()
Eoptions.headless = False

# Veri setlerini yükle
unis_last = pd.read_excel('data/unis_last_tyt.xlsx')

try:
    tyt_df = pd.read_excel('data/tyt_df.xlsx')
    processed_bolums = set()
    processed_bolums.update(tyt_df['bolum'].tolist())
    
    
except Exception as e:
    tyt_df = pd.DataFrame(columns=['yop','bolum','type','TYT Temel Matematik','TYT Fen Bilimleri','TYT Türkçe','TYT Sosyal Bilimler'])

# Ana döngü
for url in range(len(unis_last)):
    try:
        bolum = unis_last['bolum'][url]  # unis_last yapınıza göre düzenleyin
        
        # Eğer bu bölüm zaten işlenmişse atla
        if bolum in processed_bolums:
            print(f"'{bolum}' bölümü daha önce işlenmiş, atlıyorum.")
            continue
            
        driver = webdriver.Firefox(options=firefox_options)
        driver.get(unis_last['url'][url])
        time.sleep(2)
        
        try:
            ec = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="mydata"]/thead/tr[1]/th[2]'))
            )
        except:
            print(f"'{bolum}' bölümü için tablo yüklenemedi, atlıyorum.")
            driver.quit()
            continue
            
        driver.maximize_window()
        page = driver.page_source
        soup = BeautifulSoup(page, 'html.parser')
        
        type = driver.find_element(By.XPATH, '/html/body/div[1]/div[2]/div[2]/div[1]/div[1]/div/h2/strong').text
        
        last_bracket_index = type.rfind('(')
        if last_bracket_index != -1:  # Eğer parantez bulunduysa
            closing_bracket_index = type.rfind(')')
            if closing_bracket_index > last_bracket_index:  # Eğer kapanan parantez, açılan parantezden sonra geliyorsa
                # Parantezlerin içindeki metni al (parantezler hariç)
                type_in_brackets = type[last_bracket_index+1:closing_bracket_index]
        else:
            type_in_brackets = "Bilinmiyor"
        
        # TYT alanı için
        
        time.sleep(2)
        try:
            rows = driver.find_elements(By.XPATH, '//*[@id="mydata"]/tbody/tr')
            
            if len(rows) == 0 or (len(rows) == 1 and "Tabloda herhangi bir veri mevcut değil" in rows[0].text):
                print(f"'{bolum}' bölümü için Tabloda herhangi bir veri mevcut değil, atlıyorum.")
                continue
            
            has_next_page = True
            
            while has_next_page:
                rows = driver.find_elements(By.XPATH, '//*[@id="mydata"]/tbody/tr')
                row_count = len(rows)
                
                for x in range(1, row_count + 1):
                    try:
                        uni_name = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[2]/small/a').get_attribute('href').split('=')[1].strip()
                        tyt_mat = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[10]').text
                        tyt_fen = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[11]').text
                        tyt_turkce = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[8]').text
                        tyt_sosyal = driver.find_element(By.XPATH, f'//*[@id="mydata"]/tbody/tr[{x}]/td[9]').text
                        
                        new_row = pd.DataFrame([{
                            'yop': uni_name,
                            'bolum': bolum,
                            'type': type_in_brackets,
                            'TYT Temel Matematik': tyt_mat,
                            'TYT Fen Bilimleri': tyt_fen,
                            'TYT Türkçe': tyt_turkce,
                            'TYT Sosyal Bilimler': tyt_sosyal
                        }])
                        
                        tyt_df = pd.concat([tyt_df, new_row], ignore_index=True)
                        
                    except Exception as e:
                        print(f"TYT - Satır {x} işlenirken hata: {e}")
                
                try:
                    next_button = driver.find_element(By.CSS_SELECTOR, '#mydata_next')
                    if 'disabled' not in next_button.get_attribute('class'):
                        next_button.click()
                        time.sleep(2)
                    else:
                        has_next_page = False
                except:
                    has_next_page = False
        
        except Exception as e:
            print(f"TYT bölümü işleme hatası: {e}")
                
        driver.quit()
        
        # Her 10 URL'de bir veri kaydet
        if url % 5 == 0 and url > 0:
        #     say_df.to_excel("data/say_df.xlsx", index=False)
        #     soz_df.to_excel("data/soz_df.xlsx", index=False)
        #     ea_df.to_excel("data/ea_df.xlsx", index=False)
        #     dil_df.to_excel("data/dil_df.xlsx", index=False)
            tyt_df.to_excel("data/tyt_df.xlsx", index=False)
            print(f"{url} bölüm işlendi, veriler geçici olarak kaydedildi.")
            
    except Exception as e:
        print(f"URL {url} işlenirken hata oluştu: {e}")
        continue

# # Tüm işlem bittikten sonra verileri kaydet
# say_df.to_excel("data/say_df.xlsx", index=False)
# soz_df.to_excel("data/soz_df.xlsx", index=False)
# ea_df.to_excel("data/ea_df.xlsx", index=False)
# dil_df.to_excel("data/dil_df.xlsx", index=False)
tyt_df.to_excel("data/tyt_df.xlsx", index=False)

print("Tüm veriler başarıyla toplandı ve kaydedildi.")

C:\Users\batuh\AppData\Local\Temp\ipykernel_26448\2891833220.py:17: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  Eoptions.headless = False


URL 0 işlenirken hata oluştu: Message: Process unexpectedly closed with status 0

'ACİL DURUM VE AFET YÖNETİMİ (AÇIKÖĞRETİM)' bölümü için tablo yüklenemedi, atlıyorum.
'AÇIK DENİZ SONDAJ TEKNOLOJİSİ' bölümü için tablo yüklenemedi, atlıyorum.


KeyboardInterrupt: 